In [ ]:
# Using data from:
# https://www.kaggle.com/c/tabular-playground-series-apr-2021

In [ ]:
%ls
!unzip test.csv.zip
!unzip train.csv.zip

sample_data/           test.csv      train.csv
sample_submission.csv  test.csv.zip  train.csv.zip
Archive:  test.csv.zip
replace test.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [ ]:
import pandas as pd
import numpy as np
from sklearn.impute import *

train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
y = train.Survived

print(np.sum(train.PassengerId == test.PassengerId))

train.drop(['PassengerId', 'Survived', 'Name', 'Ticket'], axis=1, inplace=True)
test.drop(['PassengerId', 'Name', 'Ticket'], axis=1, inplace=True)

train.head()

0


,Pclass,Sex,Age,SibSp,Parch,Fare,Cabin,Embarked
0,1,male,NaN,2,0,27.14,C12239,S
1,3,male,NaN,0,0,13.35,NaN,S
2,3,male,0.33,1,2,71.29,NaN,S
3,3,male,19.00,0,0,13.04,NaN,S
4,3,male,25.00,0,0,7.76,NaN,S


In [ ]:
target_column = 'Age'
train[train[target_column].notnull()].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 96708 entries, 2 to 99999
Data columns (total 8 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Pclass    96708 non-null  int64  
 1   Sex       96708 non-null  object 
 2   Age       96708 non-null  float64
 3   SibSp     96708 non-null  int64  
 4   Parch     96708 non-null  int64  
 5   Fare      96579 non-null  float64
 6   Cabin     31089 non-null  object 
 7   Embarked  96460 non-null  object 
dtypes: float64(2), int64(3), object(3)
memory usage: 6.6+ MB


In [ ]:
columns_to_fit = []
for key in train.keys():
    if key == target_column: continue
    if train[target_column].notnull().sum() == train[train[target_column].notnull()][key].notnull().sum():
        columns_to_fit.append(key)

columns_to_fit

['Pclass', 'Sex', 'SibSp', 'Parch']

In [ ]:
not_append, to_append = [], []
for column in columns_to_fit:
    if train[column].dtype == np.float64:
        not_append.append(column)
    else:
        to_append.append(column)
print(not_append)
print(to_append)

[]
['Pclass', 'Sex', 'SibSp', 'Parch']


In [ ]:
from sklearn.preprocessing import *

onehotencoder = OneHotEncoder()
categorical = train[train[target_column].notnull()][to_append]
X = onehotencoder.fit_transform(categorical).toarray()

In [ ]:
X = np.concatenate((X, train[train[target_column].notnull()][not_append]), axis=1)

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor

from sklearn.metrics import mean_squared_error, accuracy_score

model = DecisionTreeRegressor()
# model = LinearRegression()
model.fit(X, train[train[target_column].notnull()][target_column])
yp = model.predict(X)

print(mean_squared_error(yp, train[train[target_column].notnull()][target_column]))

279.80903767909723


In [ ]:
Xt = onehotencoder.transform(train[to_append]).toarray()
Xt = np.concatenate((Xt, train[not_append]), axis=1)

ypt = model.predict(Xt)

In [ ]:
result = train[target_column].isnull() * ypt

In [ ]:
train.fillna({target_column: 0}, inplace=True)
train[target_column] = train[target_column] + result

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_error, accuracy_score
from sklearn.preprocessing import *

def doMethodImpute(data, target_column, model=DecisionTreeRegressor):
    class doMethodImputeIntern():
        def __init__(self, target_column, not_append, to_append, onehotencoder, model):
            self.target_column = target_column
            self.not_append = not_append
            self.to_append = to_append
            self.onehotencoder = onehotencoder
            self.model = model

        def transform(self, new_data):
            Xt = self.onehotencoder.transform(new_data[self.to_append]).toarray()
            Xt = np.concatenate((Xt, new_data[self.not_append]), axis=1)
            ypt = self.model.predict(Xt)

            result = new_data[self.target_column].isnull() * ypt
            new_data.fillna({self.target_column: 0}, inplace=True)
            new_data[self.target_column] = new_data[self.target_column] + result
            return self

    columns_to_fit = []
    for key in data.keys():
        if key == target_column: continue
        if data[target_column].notnull().sum() == data[data[target_column].notnull()][key].notnull().sum():
            columns_to_fit.append(key)

    not_append, to_append = [], []
    for column in columns_to_fit:
        if data[column].dtype == np.float64:
            not_append.append(column)
        else:
            to_append.append(column)

    onehotencoder = OneHotEncoder()
    categorical = data[data[target_column].notnull()][to_append]
    X = onehotencoder.fit_transform(categorical).toarray()
    X = np.concatenate((X, data[data[target_column].notnull()][not_append]), axis=1)

    model = model()
    model.fit(X, data[data[target_column].notnull()][target_column])
    yp = model.predict(X)

    Xt = onehotencoder.transform(data[to_append]).toarray()
    Xt = np.concatenate((Xt, data[not_append]), axis=1)
    ypt = model.predict(Xt)

    result = data[target_column].isnull() * ypt
    data.fillna({target_column: 0}, inplace=True)
    data[target_column] = data[target_column] + result
    return doMethodImputeIntern(target_column, not_append, to_append, onehotencoder, model)

In [ ]:
imputer = doMethodImpute(train, 'Age')
imputer.transform(test)

<__main__.doMethodImpute.<locals>.doMethodImputeIntern at 0x7f08b86eff50>

In [ ]:
imputer = doMethodImpute(train, 'Fare')
imputer.transform(test)

<__main__.doMethodImpute.<locals>.doMethodImputeIntern at 0x7f08b82ef6d0>

In [ ]:
train.fillna({'Cabin': "NE"}, inplace=True)
test.fillna({'Cabin': "NE"}, inplace=True)

train.fillna({'Embarked': "NE"}, inplace=True)
test.fillna({'Embarked': "NE"}, inplace=True)

In [ ]:
for data in [train, test]:
    cabins_vec = []
    for i in data['Cabin']:
        if str(i) == 'NE':
            cabins_vec.append("NE")
        else:
            cabins_vec.append(i[0])
    data['Cabin'] = pd.Series(cabins_vec)

In [ ]:
for data in [train, test]:
    embk_vec = []
    for i in data['Embarked']:
        if str(i) == 'NE':
            embk_vec.append("NE")
        else:
            embk_vec.append(i)
    data['Embarked'] = pd.Series(embk_vec)

In [ ]:
from sklearn.preprocessing import KBinsDiscretizer

enc = KBinsDiscretizer(n_bins=10, encode='onehot')
enc = enc.fit(train['Age'].to_numpy().reshape(-1, 1))
Age_discretized_train = enc.transform(train['Age'].to_numpy().reshape(-1, 1))
Age_discretized_test = enc.transform(test['Age'].to_numpy().reshape(-1, 1))
# print(Age_discretized_train)

train['Fare'] = np.log(train['Fare'])
test['Fare'] = np.log(test['Fare'])
enc = KBinsDiscretizer(n_bins=20, encode='onehot')
enc = enc.fit(train['Fare'].to_numpy().reshape(-1, 1))
Fare_discretized_train = enc.transform(train['Fare'].to_numpy().reshape(-1, 1))
Fare_discretized_test = enc.transform(test['Fare'].to_numpy().reshape(-1, 1))



In [ ]:
from sklearn.preprocessing import *
categorical_features = ['Pclass', 'Sex', 'SibSp', 'Parch', 'Cabin', 'Embarked']
cat = train[categorical_features]
cat_t = test[categorical_features]
onehotencoder = OneHotEncoder()
X = onehotencoder.fit_transform(cat).toarray()
Xt =  onehotencoder.fit_transform(cat_t).toarray()

In [ ]:
X = np.concatenate([X, Age_discretized_train.toarray(), Fare_discretized_train.toarray()], axis=1)
Xt = np.concatenate((Xt, Age_discretized_test.toarray(), Fare_discretized_test.toarray()), axis=1)

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import mean_squared_error, accuracy_score
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.svm import SVC
from sklearn.ensemble import BaggingClassifier
from sklearn.linear_model import RidgeClassifier


rf = RandomForestClassifier(n_estimators=50)
ridge = RidgeClassifier()
# bg = BaggingClassifier(base_estimator=SVC(max_iter=300), n_estimators=10)
bg = LGBMClassifier()

rf.fit(X_train, y_train)
rf_pred_train = rf.predict(X_train)
rf_pred_test = rf.predict(X_test)
print(f"RF acc: {accuracy_score(y_test, rf_pred_test)} ")

ridge.fit(X_train, y_train)
ridge_pred_train = ridge.predict(X_train)
ridge_pred_test = ridge.predict(X_test)
print(f"Ridge acc: {accuracy_score(y_test, ridge_pred_test)} ")

bg.fit(X_train, y_train)
bg_pred_train = bg.predict(X_train)
bg_pred_test = bg.predict(X_test)
print(f"BG acc: {accuracy_score(y_test, bg_pred_test)} ")



RF acc: 0.75655 
Ridge acc: 0.7769 
BG acc: 0.77905 


In [ ]:
# rf_pred_train == ridge_pred_train bg_pred_train

eq1 = rf_pred_train == ridge_pred_train
eq2 = ridge_pred_train == bg_pred_train

concordance = eq1 * eq2

np.sum(concordance)/len(X_train)

0.857875

In [ ]:
not_concordance = concordance == False

In [ ]:
len(X_train), len(y_train)

(80000, 80000)

In [ ]:
len(X_train[not_concordance]), len(y_train[not_concordance])

(11370, 11370)

In [ ]:
xgb = XGBClassifier(n_jobs=-1)
xgb.fit(X_train[not_concordance], y_train[not_concordance])

xgb_pred_train = xgb.predict(X_train)
xgb_pred_test = xgb.predict(X_test)
print(f"xgb acc: {accuracy_score(y_test, xgb_pred_test)} ")

xgb acc: 0.2252 


In [ ]:
X_train.shape

(80000, 63)

In [ ]:
new_train = np.stack([rf_pred_train, ridge_pred_train, bg_pred_train, xgb_pred_train]).T
new_test = np.stack([rf_pred_test, ridge_pred_test, bg_pred_test, xgb_pred_test]).T

In [ ]:
new_train = np.concatenate((new_train, X_train), axis=1)
new_test = np.concatenate((new_test, X_test), axis=1)

In [ ]:
xgb_final = XGBClassifier(n_jobs=-1)
xgb_final.fit(new_train, y_train)

xgb_final_pred_train = xgb_final.predict(new_train)
xgb_final_pred_test = xgb_final.predict(new_test)
print(f"xgb final acc: {accuracy_score(y_test, xgb_final_pred_test)} ")

xgb final acc: 0.75655 


In [ ]:
#BAD =(

In [ ]:
new_Xt = np.stack([rf.predict(Xt), ridge.predict(Xt), bg.predict(Xt), xgb.predict(Xt)]).T
new_Xt = np.concatenate((new_Xt, Xt), axis=1)

In [ ]:
yp = xgb_final.predict(new_Xt)

with open('pred.csv', 'w') as out, open('test.csv') as inp:
    out.write('PassengerId,Survived\n')
    inp.readline()
    for v in yp:
        line = inp.readline().split(",")[0]
        out.write(line + "," + str(int(v)) + "\n")